<h1>Importing Packages

In [241]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from folium.plugins import MarkerCluster
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns


<h1> Importing John Hopkins COVID-19 Data (Updated Daily)

In [180]:
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')



<h1> Cleaning Data

In [260]:
deaths = deaths.rename(columns={'Province/State': 'state', 'Country/Region': 'country'})
confirmed=confirmed.rename(columns={'Province/State':'state','Country/Region':'country'})
recovered=recovered.rename(columns={'Province/State':'state','Country/Region':'country'})
country=country.rename(columns={'Country_Region':'Country'})

confirmed[confirmed['country']=='US']


,state,country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
225,NaN,US,37.0902,-95.7129,1,1,2,2,5,5,...,161837,188172,213372,243616,275586,308850,337072,366667,396223,429052


In [182]:
#Stats
totalConfirmedCases = "{:,}".format(country['Confirmed'].sum())
totalDeaths="{:,}".format(country['Deaths'].sum())
totalRecovered="{:,}".format(country['Recovered'].sum())
activeCases="{:,}".format(country['Active'].sum())

print(f'Total Confirmed Cases: {totalConfirmedCases}')
print(f'Total Deaths: {totalDeaths}')
print(f'Total Recovered: {totalRecovered}')
print(f'Active Cases: {activeCases}')


Total Confirmed Cases: 1,503,900
Total Deaths: 89,931
Total Recovered: 340,112
Active Cases: 666,113


<h1> COVID-19 Cases by Countries

In [183]:
sortedCases=country.sort_values('Confirmed',ascending=False)
sortedCases=sortedCases.drop(columns=['Lat','Long_'])

#Interactive Chart that lets user input how many countries they would like to see (default 10)
#Returns countries in order from hardest hit to least hit
def show_latest_cases(n):
    n=int(n)
    return sortedCases.head(n).style.background_gradient(cmap='Blues')
#We pass n as a string to let user input a string
#Pass n as an int to get a slider
interact(show_latest_cases, n='10')

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

<function __main__.show_latest_cases(n)>

<h1> Interactive Graph that plots n worse hit countries

In [184]:
#Graph that plots the worst hit countries:
#x is country name 
#y is confirmed cases
#size is number of deaths
def chart(n):
    n=int(n)
    fig = px.scatter(sortedCases.head(n), x="Country", y="Confirmed", size="Deaths", color="Country",
               hover_name="Country",size_max=52,template="plotly_dark")
    
    
    fig.update_layout(
    title=f"Top {n} Worst hit Countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 1000
    )
    fig.show()

interact(chart, n='10')

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

<function __main__.chart(n)>

<h1>Graph that plots cases vs deaths for desired country

In [199]:
def plot_country(country):
    labels=['Confirmed Cases','Number of Deaths']
    df_list=[confirmed,deaths]
    fig=go.Figure()
    
    for i, df in enumerate(df_list):
        x_data = np.array(df.iloc[:,5:].columns)
        if country=='World':
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis=0)
        else:
            temp=df[df['country'] == country].iloc[:,4:]
            y_data = np.sum(temp,axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data,
                                 y=y_data,
                                 mode='lines',
                                 name=labels[i]
                                ))
    
    fig.update_layout(
        title="COVID 19 Confirmed cases in " + country,
        xaxis_title='Date',
        yaxis_title='Number of Confirmed Cases',
        legend_title='<b> Legend </b>',
        template="plotly_dark",
        width = 975,
        
    )
    fig.show()
#Enter the name of your country or world for total cases
interact(plot_country, country='World')


interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

<function __main__.plot_country(country)>

In [261]:
#Interactive map that plots confirmed cases as well as displays 
worldMap=folium.Map(location=[11,0],tiles='cartodbdark_matter',zoom_start=2,max_zoom=6,min_zoom=2)
marker_cluster = MarkerCluster().add_to(worldMap)

for i in range(len(confirmed)):
    folium.Circle(
        location=[confirmed.iloc[i]['Lat'],confirmed.iloc[i]['Long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        fill_color='red',
        color='black',
        tooltip = "Country:{}<br> State: {}<br>Cases:{}<br>Deaths:{}<br>".format(confirmed["country"][i], confirmed["state"][i],confirmed.iloc[i,-1],deaths.iloc[i,-1])
        ).add_to(worldMap)  
worldMap